# Aufgabe 1 (5P)

## a)
Nutzen Sie vom Datensatz „pvtest.csv“ die Werte 'Edaily','Dci','Dcp','Dcu','Temp1','hour‘ und erstellen Sie einen DataFrame
als Zeitreihe, ähnlich wie unten abgebildet. Auf Grundlage dieser Daten soll der Energieertrag Dcp immer in 5 Minuten in
der Zukunft prognostiziert werden. (0.5P)

<img src='images/1a.png' width=400>

In [85]:
from math import ceil

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('pvtest.csv', index_col=1, parse_dates=True)
df = df[['Edaily', 'Dci', 'Dcp', 'Dcu', 'Temp1']]
df['hour'] = df.index.hour

print(df.tail(15))
print(df.shape)

                     Edaily       Dci  Dcp  Dcu  Temp1  hour
Time                                                        
2021-01-26 16:25:00   3.108  0.192504  113  587   14.0    16
2021-01-26 16:30:00   3.122  0.111913   62  554   14.0    16
2021-01-26 16:35:00   3.130  0.053846   28  520   14.0    16
2021-01-26 16:40:00   3.137  0.041176   21  510   14.0    16
2021-01-26 16:45:00   3.141  0.024283   11  453   14.0    16
2021-01-26 16:50:00   3.144  0.000000    0  271   14.0    16
2021-01-26 16:55:00   3.145  0.000000    0  257   13.0    16
2021-01-26 17:00:00   3.145  0.000000    0  251   12.0    17
2021-01-26 17:05:00   3.145  0.000000    0  250   12.0    17
2021-01-26 17:10:00   3.145  0.000000    0   37    1.0    17
2021-01-26 17:15:00   3.145  0.000000    0    0    0.0    17
2021-01-26 17:20:00   3.145  0.000000    0    0    0.0    17
2021-01-26 17:25:00   3.145  0.000000    0    0    0.0    17
2021-01-26 17:35:00   3.145  0.000000    0    0    0.0    17
2021-01-26 17:40:00   3.

C:\Users\aaron\AppData\Local\Temp\ipykernel_18612\2042717557.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('pvtest.csv', index_col=1, parse_dates=True)


## b)
Teilen Sie die Daten sinnvoll in Lern-und Testdatensatz auf, so dass die zeitlichen Reihenfolgen bestehen bleiben ! (0.5P)

In [86]:
end_train = ceil(len(df) * 0.9)
df_train = df.iloc[:end_train]
df_test = df.iloc[end_train:]

X_train = df_train
X_test = df_test
y_train = df_train[['Dcp']]
y_test  = df_test[['Dcp']]

print('Training attribute shape:', X_train.shape, 'Training label shape:', y_train.shape)
print('Test attribute shape:', X_test.shape, 'Test label shape:', y_test.shape)

Training attribute shape: (3191, 6) Training label shape: (3191, 1)
Test attribute shape: (354, 6) Test label shape: (354, 1)


## c)
Standardisieren Sie die Daten mit dem Standardscaler und restrukturieren Sie die Zeitreihe in einzelne Datenfenster, so dass
Sie aus allen vorhandenen 6 Features aus den letzten drei Stunden immer jeweils den Energieertrag für die nächsten 5
Minuten vorhersagen können. (1,5P)

In [87]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = scaler.fit_transform(y_train)
y_test = scaler.transform(y_test)

def restructure_data(x:pd.DataFrame, y:pd.DataFrame, window:int, horizon:int):
    x_, y_ = [], []
    idx_range = range(len(x) - (window + horizon))
    for i in idx_range:
        x_.append(x[i:i + window])
        y_.append(y[i + window + horizon])
    x_ = np.array(x_)
    y_ = np.array(y_)
    return x_, y_

X_train, y_train = restructure_data(X_train, y_train, window=36, horizon=1)
X_test, y_test = restructure_data(X_test, y_test, window=36, horizon=1)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(3154, 36, 6) (3154, 1) (317, 36, 6) (317, 1)


# d)
Definieren und kompilieren Sie jetzt ein passendes Modell in Keras, welches mind. eine LSTM-Layer nutzt und DropoutLayer einfügt, um „Überanpassung“ zu verhindern. (1P)

## e)
Trainieren Sie das Modell mit allen Lerndaten über mind. 100 Epochen, nutzen Sie ggf. auch callbacks, um den Lernprozess
zu verkürzen. Verbessern Sie Ihr Training und Ihre Modellarchitektur so lange, bis Sie einen Mean absolute Error von mind.
50 erzielen! (1P)

# f)
Visualisieren Sie Ihre Prognoseergebnisse, so ähnlich wie unten angegeben im Vergleich gemessener Daten mit
prognostizierten Werten! (0.5P)

<img src="images/1f.png" width=900>